# Neural Networks
This project implements a simple neural network. The purpose is to demonstrate the basic structure. The network is tested on the [MNIST handwritten digit data set](http://yann.lecun.com/exdb/mnist/). This exercise is inspired by Andrew Ng's Deep Learning series on Coursera. Dr. Ng suggests that we can best understand deep learning by implementing algorithms ourselves.

First we import some libraries and load the data.

In [3]:
import numpy as np
import pandas as pd
import math
from PIL import Image
import idx2numpy
import os

Next we load the MNIST handwritten digit data set. The training set consists of 60,000 examples and the test set has 10,000 examples. Each example is a 28X28 greyscale image, formatted as a two-dimensional array with values ranging from 0 to 255, corresponding to black and white pixels respectively.

It is necessary for every training and test example to be treated as a one-dimensional array, and so we reshape the examples into arrays with 28 * 28 = 784 elements.

In the given data sets, the output value is an integer from 0 to 9, indicating the digit. We reshape the outputs using a One Hot Encoding to an array with all zeroes except for a one in the position of the value of the digit. For example, 3 is represented as [0,0,0,1,0,0,0,0,0,0].

In [4]:
# Load data
train_X = idx2numpy.convert_from_file("train-images-idx3-ubyte")
train_y = idx2numpy.convert_from_file("train-labels-idx1-ubyte")
test_X  = idx2numpy.convert_from_file("t10k-images-idx3-ubyte")
test_y  = idx2numpy.convert_from_file("t10k-labels-idx1-ubyte")
# Reshape the inputs so there is a 1D array of features for every input.
train_X = train_X.reshape(train_X.shape[0],train_X.shape[1]*train_X.shape[2])
test_X  = test_X.reshape(test_X.shape[0],test_X.shape[1]*test_X.shape[2])
# Encode the training and test y values in a manner that is easier to use in the cost function
train_y_ohe = np.zeros((len(train_y),10))
train_y_ohe[np.arange(len(train_y)), train_y] = 1
test_y_ohe = np.zeros((len(test_y),10))
test_y_ohe[np.arange(len(test_y)), test_y] = 1

Now we set the model hyperparameters. In this implementation, there are four hyperparameters to set: the learning rate, the number of hidden layers, the size of the hidden layers, and the number of iterations.

It appears, through experimentation, that it is necessarily to lower the learning rate as the training of the model progresses. Consequently, the learning rate hyperparameter is treated as a function of the iteration number rather than a fixed value.

In [29]:
# Network parameters
def learning_rate(iter_num):
    return min(0.01, 0.05/math.sqrt(iter_num+1))
# Layer sizes. The array can be any length.
# It must start with train_X.shape[1] (number of inputs) and end with 10 (number of digits)
layer_sizes = [train_X.shape[1],100, 50, 10]
num_iterations = 150 # Number of full (forward and back propagation) iterations of the network

Next we define the activation functions. All layers will use a rectified linear unit (ReLU) as the activation, except for the output layer, which uses a softmax function.

For backpropagation, we need to calculate the derivative of the ReLU function as well, given as dReLU().

In [6]:
# a is a vector
def ReLU(a):
    return np.maximum(a,0)
def dReLU(a):
    return (a>0).astype(int)
def softmax(a):
    return np.exp(a) / np.sum(np.exp(a),axis=1,keepdims=True)

Now we construction a randomized initial network. The weights W should not all start as zero, since in that case it will be impossible for backpropagation to break symmetry, and the network would essentially collapse into a single node for each layer.

In [7]:
def random_init_network(X):
    W = [0]+[np.random.randn(layer_sizes[i],layer_sizes[i+1])*0.01 for i in range(len(layer_sizes)-1)]
    b = [0]+[np.zeros((1,layer_sizes[i+1])) for i in range(len(layer_sizes)-1)]
    W[0] = X
    return W,b

Forward propagation. Given a network W,b and inputs X, determine the predicted and intermediate network values. The predictions are A[len(layer_sizes)-1] (the final entry of the array A). The intermediate values are returned because they are necessary in the calculation of derivatives in the backpropagation stage.

In [8]:
# Apply forward propagation on a set of training examples
def forward_prop(X,W,b):
    A = [X]
    Z = [0]
    A_cur = X
    for i in range(len(layer_sizes)-1):
        Z_cur = np.add(np.matmul(A_cur,W[i+1]),b[i+1])
        if i < len(layer_sizes)-2:
            A_cur = ReLU(Z_cur)
        else:
            A_cur = softmax(Z_cur)
        A.append(A_cur)
        Z.append(Z_cur)
    return A,Z

Define the cost fuction. The cost is a measure of how far the predicted values y_pred are from the actual values y. A lower cost indicates a more accurate network. We use a cross-entropy cost function.

In [9]:
# Calculate the cost function, using predicted and actual y values
def cost(y_pred, y):
    return -1./len(y) * np.sum(np.add( np.multiply(y, np.log(y_pred)), np.multiply(1-y,np.log(1-y_pred)) ))

Calculate the derivatives of difference between predicted and actual values, with respect to network parameters W and b.

In [10]:
# Calculate derivatives to be used in backprop
def calc_derivatives(X,W,b,A,Z,y):
    dZ = [0 for i in range(len(layer_sizes))]
    dW = [0 for i in range(len(layer_sizes))]
    db = [0 for i in range(len(layer_sizes))]

    dZ[len(layer_sizes)-1] = A[len(layer_sizes)-1]-y
    dW[len(layer_sizes)-1] = 1./len(X)*np.matmul(A[len(layer_sizes)-2].T,dZ[len(layer_sizes)-1])
    db[len(layer_sizes)-1] = 1./len(X)*np.sum(dZ[len(layer_sizes)-1],axis=0,keepdims=True)

    for layer in range(len(layer_sizes)-2,0,-1):
        dZ[layer] = np.multiply( np.matmul(dZ[layer+1],W[layer+1].T), dReLU(Z[layer]) )
        dW[layer] = 1./len(X) * np.matmul(A[layer-1].T,dZ[layer])
        db[layer] = 1./len(X) * np.sum(dZ[layer],axis=0,keepdims=True)
    return dZ, dW, db

Backpropagation. Here we adjust the network parameters W and b.

In [11]:
def backprop(X,W,b,A,Z,y,iter_num):
    dZ, dW, db = calc_derivatives(X,W,b,A,Z,y)
    for i in range(1,len(layer_sizes)):
        W[i] -= learning_rate(iter_num) * dW[i]
        b[i] -= learning_rate(iter_num) * db[i]
    return W,b

In [12]:
#def nn_iterate(X,W,b,y):
#    A,Z = forward_prop(X,W,b)
#    W,b = backprop(X,W,b,A,Z,y)
#    return W,b

Build and train a neural network. Each iteration performs a forward and backpropagation step. The result should be a trained network that reasonably accurately identifies digits.

In [13]:
def build_network(X,y):
    W,b = random_init_network(X)
    for i in range(num_iterations):
        A,Z = forward_prop(X,W,b)
        W,b = backprop(X,W,b,A,Z,y,i)
        if (i+1)%5 == 0:
            print "Iteration " + str(i+1) + ": " + str(cost(A[len(layer_sizes)-1],y))
    return W,b

Now let's train the network. This step might take a few minutes.

In [27]:
W,b = build_network(train_X,train_y_ohe)

Iteration 5: 3.22346182847
Iteration 10: 3.15871947487
Iteration 15: 2.98037059772
Iteration 20: 2.54447874134
Iteration 25: 2.0300854004
Iteration 30: 5.34294014897
Iteration 35: 2.46298843312
Iteration 40: 2.26037498586
Iteration 45: 2.06350126234
Iteration 50: 1.80896355152
Iteration 55: 1.50904240143
Iteration 60: 1.31253710244
Iteration 65: 1.70890548549
Iteration 70: 1.18816506597
Iteration 75: 1.05986324879
Iteration 80: 0.982870058645
Iteration 85: 0.927800305229
Iteration 90: 0.885179649291
Iteration 95: 0.850777075098
Iteration 100: 0.822188582512
Iteration 105: 0.797907527346
Iteration 110: 0.776927269238
Iteration 115: 0.758485900509
Iteration 120: 0.742061600961
Iteration 125: 0.727317480478
Iteration 130: 0.713952496958
Iteration 135: 0.701764270243
Iteration 140: 0.690570302303
Iteration 145: 0.680229561968
Iteration 150: 0.670620383628


Now let's see how it performs. We will apply the network trained above on the test set and report the accuracy. The value will vary because of randomness in the initial weights, but it should be around 0.9, meaning that about 90% of examples in the test set are correctly classified.

In [28]:
def test_network(X,W,b,y):
    A,Z = forward_prop(X,W,b)
    y_pred = A[-1]
    y_pred = [np.where(y_pred[i]==max(y_pred[i]))[0][0] for i in range(len(y_pred))]
    return y_pred
y_pred = test_network(test_X,W,b,test_y_ohe)
accuracy = len([i for i in range(len(y_pred)) if y_pred[i] == test_y[i]]) / float(len(y_pred))
print "Accuracy is " + str(accuracy)

Accuracy is 0.8901


### Observations
The network's performance of about 90% suggests that the algorithm is working, though with a considerably worse performance than the algorithms reported by LeCun et al.

Little work was done to optimize the hyperparameters. If a more systematic hyperparameter search was undertaken, it would be appropriate to divide the training set into training and dev sets, to use the dev set to optimize hyperparameters, and to use the test set to evaluate network performance.